In [47]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [48]:
data = pd.read_csv("GettingData/gameLogs.csv")

In [49]:
data

,Unnamed: 0,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,...,GAME_DATE,GAME_ID,HOME_FLAG,AWAY_FLAG,HOME_WIN_PCTG,AWAY_WIN_PCTG,TOTAL_WIN_PCTG,ROLLING_SCORING_MARGIN,ROLLING_OE,NUM_REST_DAYS
0,0,Atlanta,Hawks,1610612737,1,0,1,0,0,0,...,2021-05-16,22001066,1,0,0.694444,0.444444,0.569444,18.666667,0.594231,3.0
1,1,OPPONENTS,Rockets,1610612745,0,1,0,0,0,1,...,2021-05-16,22001066,0,1,0.250000,0.222222,0.236111,-8.000000,0.559973,2.0
2,2,Atlanta,Hawks,1610612737,1,0,1,0,0,0,...,2021-05-13,22001049,1,0,0.685714,0.444444,0.563380,9.333333,0.571405,1.0
3,3,OPPONENTS,Magic,1610612753,0,1,0,0,0,1,...,2021-05-13,22001049,0,1,0.305556,0.294118,0.300000,-22.333333,0.502823,2.0
4,4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,...,2021-05-12,22001042,1,0,0.676471,0.444444,0.557143,-0.666667,0.589472,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,7075,OPPONENTS,Pistons,1610612765,0,1,0,0,0,1,...,2023-02-27,22200919,0,1,0.266667,0.218750,0.241935,-5.666667,0.529425,2.0
7076,7076,Detroit,Pistons,1610612765,1,0,1,0,0,0,...,2023-02-03,22200785,1,0,0.269231,0.259259,0.264151,-1.000000,0.545693,4.0
7077,7077,OPPONENTS,Hornets,1610612766,0,1,0,0,0,1,...,2023-02-03,22200785,0,1,0.304348,0.258065,0.277778,-10.333333,0.524888,1.0
7078,7078,Charlotte,Hornets,1610612766,0,1,0,1,0,0,...,2022-12-14,22200414,1,0,0.285714,0.214286,0.250000,-14.666667,0.530214,3.0


In [50]:
def preprocess_data(og_data, feature_set):
        data = pd.read_csv(og_data)
        data.dropna()
        data["TS_PCT"] = data["PTS"]/(2*(data["FGA"]+(0.475*data["FTA"])))
        data["STL+BLK"] = data["STL"] + data["BLK"]
        data["TOV_PCT"] = data["TOTAL_TURNOVERS"]/(data["FGA"]+(0.475*data["FTA"])+data["AST"]+data["TOTAL_TURNOVERS"])
        data['LAST_GAME_OE'] = data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].shift(1)
        data['LAST_GAME_HOME_WIN_PCTG'] = data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_WIN_PCTG'].shift(1)
        data['LAST_GAME_AWAY_WIN_PCTG'] = data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_WIN_PCTG'].shift(1)
        data['LAST_GAME_TOTAL_WIN_PCTG'] = data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_WIN_PCTG'].shift(1)
        data['LAST_GAME_ROLLING_SCORING_MARGIN'] = data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_SCORING_MARGIN'].shift(1)
        data['LAST_GAME_ROLLING_OE'] = data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_OE'].shift(1)
        data['LAST_3_GAME_AVG_OE'] = (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].shift(1) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].shift(2) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].shift(3))/3
        data['LAST_3_GAME_AVG_HOME_WIN_PCTG'] = (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_WIN_PCTG'].shift(1) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_WIN_PCTG'].shift(2) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_WIN_PCTG'].shift(3))/3
        data['LAST_3_GAME_AVG_AWAY_WIN_PCTG'] = (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_WIN_PCTG'].shift(1) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_WIN_PCTG'].shift(2) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_WIN_PCTG'].shift(3))/3
        data['LAST_3_GAME_AVG_TOTAL_WIN_PCTG'] = (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_WIN_PCTG'].shift(1) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_WIN_PCTG'].shift(2) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_WIN_PCTG'].shift(3))/3
        data['LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN'] = (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_SCORING_MARGIN'].shift(1) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_SCORING_MARGIN'].shift(2) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_SCORING_MARGIN'].shift(3))/3
        data['LAST_3_GAME_AVG_ROLLING_OE'] = (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_OE'].shift(1) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_OE'].shift(2) + data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_OE'].shift(3))/3
        data["LAST_3_GAME_AVG_TS_PCT"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TS_PCT'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TS_PCT'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TS_PCT'].shift(3)))/3
        data["LAST_3_GAME_AVG_FG3_PCT"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FG3_PCT'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FG3_PCT'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FG3_PCT'].shift(3)))/3
        data["LAST_3_GAME_AVG_TOT_REB"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOT_REB'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOT_REB'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOT_REB'].shift(3)))/3
        data["LAST_3_GAME_AVG_STL+BLK"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['STL+BLK'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['STL+BLK'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['STL+BLK'].shift(3)))/3
        data["LAST_3_GAME_AVG_TOV_PCT"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOV_PCT'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOV_PCT'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOV_PCT'].shift(3)))/3
        data["LAST_3_GAME_AVG_PTS"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['PTS'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['PTS'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['PTS'].shift(3)))/3
        data["LAST_3_GAME_AVG_FG_PCT"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FG_PCT'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FG_PCT'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FG_PCT'].shift(3)))/3
        data["LAST_3_GAME_AVG_FT_PCT"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FT_PCT'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FT_PCT'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['FT_PCT'].shift(3)))/3
        data["LAST_3_GAME_AVG_AST"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AST'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AST'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AST'].shift(3)))/3
        data["LAST_3_GAME_AVG_STL"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['STL'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['STL'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['STL'].shift(3)))/3
        data["LAST_3_GAME_AVG_TOTAL_TURNOVERS"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_TURNOVERS'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_TURNOVERS'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_TURNOVERS'].shift(3)))/3
        data["LAST_3_GAME_AVG_BLK"] = ((data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['BLK'].shift(1)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['BLK'].shift(2)) + (data.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['BLK'].shift(3)))/3
        gameDF = data
        gamedf = data
        homeTeamFrame = gameDF[gameDF['CITY'] != 'OPPONENTS']
        awayTeamFrame = gamedf[gamedf['CITY'] == 'OPPONENTS']
        if feature_set == "REG":
            homeTeamFrame = homeTeamFrame[['LAST_3_GAME_AVG_OE','LAST_3_GAME_AVG_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_3_GAME_AVG_AWAY_WIN_PCTG','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','W','TEAM_ID','GAME_ID','GAME_DATE','SEASON',"LAST_3_GAME_AVG_FG_PCT","LAST_3_GAME_AVG_FG3_PCT","LAST_3_GAME_AVG_FT_PCT","LAST_3_GAME_AVG_ROLLING_OE","LAST_3_GAME_AVG_TOT_REB","LAST_3_GAME_AVG_AST","LAST_3_GAME_AVG_STL","LAST_3_GAME_AVG_TOTAL_TURNOVERS","LAST_3_GAME_AVG_BLK","LAST_3_GAME_AVG_PTS"]]
            awayTeamFrame = awayTeamFrame[['LAST_3_GAME_AVG_OE','LAST_3_GAME_AVG_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_3_GAME_AVG_AWAY_WIN_PCTG','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','TEAM_ID','GAME_ID','SEASON',"LAST_3_GAME_AVG_FG_PCT","LAST_3_GAME_AVG_FG3_PCT","LAST_3_GAME_AVG_FT_PCT","LAST_3_GAME_AVG_ROLLING_OE","LAST_3_GAME_AVG_TOT_REB","LAST_3_GAME_AVG_AST","LAST_3_GAME_AVG_STL","LAST_3_GAME_AVG_TOTAL_TURNOVERS","LAST_3_GAME_AVG_BLK","LAST_3_GAME_AVG_PTS"]]
        elif feature_set == "CF":
            homeTeamFrame = homeTeamFrame[['LAST_3_GAME_AVG_OE','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','W','TEAM_ID','GAME_ID','GAME_DATE','SEASON',"LAST_3_GAME_AVG_TS_PCT","LAST_3_GAME_AVG_FG3_PCT","LAST_3_GAME_AVG_PTS"]]
            awayTeamFrame = awayTeamFrame[['LAST_3_GAME_AVG_OE','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','TEAM_ID','GAME_ID','SEASON',"LAST_3_GAME_AVG_TS_PCT","LAST_3_GAME_AVG_TOTAL_TURNOVERS","LAST_3_GAME_AVG_PTS"]]
        elif feature_set == "FE":
            homeTeamFrame = homeTeamFrame[['LAST_3_GAME_AVG_OE','NUM_REST_DAYS','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','W','TEAM_ID','GAME_ID','GAME_DATE','SEASON',"LAST_3_GAME_AVG_TS_PCT","LAST_3_GAME_AVG_FG3_PCT","LAST_3_GAME_AVG_TOT_REB","LAST_3_GAME_AVG_STL+BLK","LAST_3_GAME_AVG_TOV_PCT","LAST_3_GAME_AVG_PTS"]]
            awayTeamFrame = awayTeamFrame[['LAST_3_GAME_AVG_OE','NUM_REST_DAYS','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','TEAM_ID','GAME_ID','SEASON',"LAST_3_GAME_AVG_TS_PCT","LAST_3_GAME_AVG_FG3_PCT","LAST_3_GAME_AVG_TOT_REB","LAST_3_GAME_AVG_STL+BLK","LAST_3_GAME_AVG_TOV_PCT","LAST_3_GAME_AVG_PTS"]]
        elif feature_set == "OG":
            homeTeamFrame = homeTeamFrame[['LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_GAME_AWAY_WIN_PCTG','LAST_GAME_TOTAL_WIN_PCTG','LAST_GAME_ROLLING_SCORING_MARGIN','LAST_GAME_ROLLING_OE','W','TEAM_ID','GAME_ID','GAME_DATE','SEASON']]
            awayTeamFrame = awayTeamFrame[['LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_GAME_AWAY_WIN_PCTG','LAST_GAME_TOTAL_WIN_PCTG','LAST_GAME_ROLLING_SCORING_MARGIN','LAST_GAME_ROLLING_OE','TEAM_ID','GAME_ID','SEASON']]
        elif feature_set == "OG3":
            homeTeamFrame = homeTeamFrame[['LAST_3_GAME_AVG_OE','LAST_3_GAME_AVG_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_3_GAME_AVG_AWAY_WIN_PCTG','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','W','TEAM_ID','GAME_ID','GAME_DATE','SEASON']]
            awayTeamFrame = awayTeamFrame[['LAST_3_GAME_AVG_OE','LAST_3_GAME_AVG_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_3_GAME_AVG_AWAY_WIN_PCTG','LAST_3_GAME_AVG_TOTAL_WIN_PCTG','LAST_3_GAME_AVG_ROLLING_SCORING_MARGIN','LAST_3_GAME_AVG_ROLLING_OE','TEAM_ID','GAME_ID','SEASON']]
        colRenameDict = {}
        for col in homeTeamFrame.columns:
            if (col != 'GAME_ID') & (col != 'SEASON') & (col != 'GAME_DATE'):
                colRenameDict[col] = 'HOME_' + col 
        homeTeamFrame.rename(columns=colRenameDict,inplace=True)
        colRenameDict = {}
        for col in awayTeamFrame.columns:
            if (col != 'GAME_ID') & (col != 'SEASON') & (col != 'GAME_DATE'):
                colRenameDict[col] = 'AWAY_' + col 
        awayTeamFrame.rename(columns=colRenameDict,inplace=True)
        data = pd.merge(homeTeamFrame, awayTeamFrame, how="inner", on=["GAME_ID","SEASON"]).drop(['GAME_ID','AWAY_TEAM_ID','HOME_TEAM_ID'],axis=1)
        return data

In [51]:
data = preprocess_data("GettingData/gameLogs.csv", "OG")

In [52]:
df = data

In [53]:
df

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,GAME_DATE,SEASON,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE
0,0.555556,0.685714,3.0,0.444444,0.563380,9.333333,0.571405,1,2021-05-16,2020-21,0.612903,0.250000,2.0,0.228571,0.239437,-2.000000,0.605315
1,0.573913,0.676471,1.0,0.444444,0.557143,-0.666667,0.589472,1,2021-05-13,2020-21,0.500000,0.305556,2.0,0.303030,0.304348,-18.000000,0.512228
2,0.584746,0.666667,2.0,0.444444,0.550725,8.666667,0.640592,1,2021-05-12,2020-21,0.612403,0.500000,2.0,0.428571,0.463768,0.666667,0.567718
3,0.609756,0.656250,4.0,0.444444,0.544118,11.333333,0.653327,1,2021-05-10,2020-21,0.539568,0.500000,2.0,0.441176,0.470588,0.666667,0.570866
4,0.512397,0.354839,1.0,0.558824,0.461538,11.000000,0.596498,1,2021-05-06,2020-21,0.727273,0.656250,1.0,0.457143,0.552239,17.333333,0.633081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.531250,1.000000,2.0,0.500000,0.666667,-0.333333,0.547383,1,2022-10-25,2022-23,0.533898,1.000000,3.0,0.000000,0.333333,-9.666667,0.549413
3536,0.626087,0.242424,2.0,0.212121,0.227273,-10.666667,0.564607,0,2023-03-09,2022-23,0.605263,0.354839,2.0,0.277778,0.313433,-6.666667,0.534304
3537,0.517857,0.357143,2.0,0.264706,0.306452,7.666667,0.553343,1,2023-02-27,2022-23,0.537815,0.266667,2.0,0.225806,0.245902,-8.000000,0.538467
3538,0.529412,0.240000,4.0,0.259259,0.250000,-0.333333,0.566698,1,2023-02-03,2022-23,0.522523,0.304348,1.0,0.266667,0.283019,-6.666667,0.550896


In [54]:
df = df.drop("SEASON", axis = 1)

In [55]:
df

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE
0,0.555556,0.685714,3.0,0.444444,0.563380,9.333333,0.571405,1,2021-05-16,0.612903,0.250000,2.0,0.228571,0.239437,-2.000000,0.605315
1,0.573913,0.676471,1.0,0.444444,0.557143,-0.666667,0.589472,1,2021-05-13,0.500000,0.305556,2.0,0.303030,0.304348,-18.000000,0.512228
2,0.584746,0.666667,2.0,0.444444,0.550725,8.666667,0.640592,1,2021-05-12,0.612403,0.500000,2.0,0.428571,0.463768,0.666667,0.567718
3,0.609756,0.656250,4.0,0.444444,0.544118,11.333333,0.653327,1,2021-05-10,0.539568,0.500000,2.0,0.441176,0.470588,0.666667,0.570866
4,0.512397,0.354839,1.0,0.558824,0.461538,11.000000,0.596498,1,2021-05-06,0.727273,0.656250,1.0,0.457143,0.552239,17.333333,0.633081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.531250,1.000000,2.0,0.500000,0.666667,-0.333333,0.547383,1,2022-10-25,0.533898,1.000000,3.0,0.000000,0.333333,-9.666667,0.549413
3536,0.626087,0.242424,2.0,0.212121,0.227273,-10.666667,0.564607,0,2023-03-09,0.605263,0.354839,2.0,0.277778,0.313433,-6.666667,0.534304
3537,0.517857,0.357143,2.0,0.264706,0.306452,7.666667,0.553343,1,2023-02-27,0.537815,0.266667,2.0,0.225806,0.245902,-8.000000,0.538467
3538,0.529412,0.240000,4.0,0.259259,0.250000,-0.333333,0.566698,1,2023-02-03,0.522523,0.304348,1.0,0.266667,0.283019,-6.666667,0.550896


In [56]:
excluded_col = df[['HOME_W','GAME_DATE']]


In [57]:
df_to_scale = df.drop(['HOME_W','GAME_DATE'], axis=1)

In [58]:
scaler = StandardScaler()

In [59]:
scaled_data = scaler.fit_transform(df_to_scale)

In [60]:
scaled_df = pd.DataFrame(scaled_data, columns=df_to_scale.columns)

In [61]:
final_df = pd.concat([scaled_df, excluded_col], axis=1)

In [62]:
test = final_df[(final_df['GAME_DATE'] >= "2021-05-01") & (final_df['GAME_DATE'] <= "2022-06-03")]

In [63]:
print(test)

      HOME_LAST_GAME_OE  HOME_LAST_GAME_HOME_WIN_PCTG  HOME_NUM_REST_DAYS  \
0             -0.176107                      0.648150            0.868742   
1              0.168292                      0.601843           -1.166081   
2              0.371521                      0.552730           -0.148670   
3              0.840731                      0.500548            1.886154   
4             -0.985795                     -1.009374           -1.166081   
...                 ...                           ...                 ...   
2305          -0.429366                      1.596378           -0.148670   
2306          -0.436671                      0.552730            0.868742   
2307           1.532256                     -0.195817           -0.148670   
2308          -0.055515                     -1.302643           -1.166081   
2309           0.833581                      0.433456           -0.148670   

      HOME_LAST_GAME_AWAY_WIN_PCTG  HOME_LAST_GAME_TOTAL_WIN_PCTG  \
0     